In [7]:
from selenium import webdriver

from selenium.webdriver.common.by import By

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [54]:
driver = webdriver.Chrome()
driver.get("https://dhcappl.nic.in/FreeText/launchbrowsejud.do#")
driver.maximize_window()

In [101]:
data = []
final_data = []
years = ['2012','2013', '2014', '2015']
def court_data(driver, year):
    
    # Locate and click the link for the specific year
    page_year = driver.find_element(By.XPATH, f"""//a[@onclick="setiframe('{year}.html');"]""")
    page_year.click()
    
    # switch to the iframe which contains the data
    iframe = driver.find_element(By.XPATH, '//iframe[@id="judiframeid"]')
    driver.switch_to.frame(iframe)
    
    
    # Extract row from table
    rows = driver.find_elements(By.XPATH,'//tbody[@id="data"]/tr')
    
    # printing year after each iteration
    print("Scraping for year:", year) 
    
    #Iterate through each row in rows and extract data
    for row in rows:
        
        td_elements = row.find_elements(By.XPATH, './/td')
        
        row_data = [td.text for td in td_elements]
        
        # Extracting the link which is in the 4 element.
        link = td_elements[4].find_element(By.XPATH, './/a').get_attribute('href')
        row_data.append(link)
        
        data.append(row_data)
        
    #Switch back the original content    
    driver.switch_to.default_content()
    
    return data

for year in years:
    final_data.extend(court_data(driver, year))

Scraping for year: 2012
Scraping for year: 2013
Scraping for year: 2014
Scraping for year: 2015


In [102]:
driver.quit()

In [103]:
df_list = []
for list_ in data:
    new_row = {'CASE NO': list_[0], 'CATEGORY': list_[1], 'PARTY NAME': list_[2], 'JUDGEMENT DATE': list_[3], 'JUDGEMENT PDF LINK':list_[5]}
    df_list.append(new_row)
    
df = pd.DataFrame(df_list)
df.head()

,CASE NO,CATEGORY,PARTY NAME,JUDGEMENT DATE,JUDGEMENT PDF LINK
0,W.P.(C) 3503/1992,MISCELLANEOUS DDA CASES (NOT COVERED BY ANY AB...,RATI RAM AND ANR. VS DELHI DEVELOPMENT AUTHORI...,02-02-12,https://dhccaseinfo.nic.in/jupload/dhc/HKO/jud...
1,W.P.(C) 198/1995,LABOUR AND INDUSTRIAL LAW MATTERS,M/S LAKSHMI TYPE FOUNDARY & ORS. VS REGIONAL P...,11-10-12,https://dhccaseinfo.nic.in/jupload/dhc/MUG/jud...
2,W.P.(C) 3466/1995,"INDUSTRIAL DISPUTES ACT, 1947 - CASES RELATING...",ARUN KUMAR VS MANAGEMENT OF CABLE CORUM OF INDIA,24-09-12,https://dhccaseinfo.nic.in/jupload/dhc/MUG/jud...
3,W.P.(C) 4599/1995,PENSION MATTERS,SK GUPTA VS AIRPORT AUTHORITY OF INDIA,20-12-12,https://dhccaseinfo.nic.in/jupload/dhc/MUG/jud...
4,W.P.(C) 773/1995,Matters against orders of the Board of Revenue...,R.N. SAHNI & ANR. VS THE FINANCIAL COMMISSIONE...,10-04-12,https://dhccaseinfo.nic.in/jupload/dhc/SUG/jud...


In [105]:
df.sample()

,CASE NO,CATEGORY,PARTY NAME,JUDGEMENT DATE,JUDGEMENT PDF LINK
19438,W.P.(C) 6803/2013,CONSTITUTIONAL VALIDITY MATTERS [CIVIL],NATIONAL BUILDING CONSTRUCTION CIROIRATUIB KUN...,09-01-14,https://dhccaseinfo.nic.in/jupload/dhc/SID/jud...


In [106]:
data = df.to_csv("C:/Users/ADMIN/Desktop/New folder/Project/Court scrape/Court_data1.csv")

In [113]:
df1 = pd.read_csv("C:/Users/ADMIN/Desktop/New folder/Project/Court scrape/Court_data1.csv")
df1.sample()

,Unnamed: 0,CASE NO,CATEGORY,PARTY NAME,JUDGEMENT DATE,JUDGEMENT PDF LINK
30390,30390,EFA(OS) 27/2015,OTHER MATTERS RELATING TO SPECIFIC PERFORMANCE,CENTRAL DAIRY FARM & ANR VS M/S PACKRAFT INDIA...,29-10-15,https://dhccaseinfo.nic.in/jupload/dhc/ISM/jud...


In [114]:
df = df1.copy()

In [119]:
df.rename(columns={'Unnamed: 0': 'S.NO'}, inplace=True)

In [120]:
df.sample()

,S.NO,CASE NO,CATEGORY,PARTY NAME,JUDGEMENT DATE,JUDGEMENT PDF LINK
31856,31856,CRL.M.C. 4606/2015,CRL.M.C. RELATING TO QUASHING OF F.I.R./CHARGE...,MANISH KUMAR KWATRA & ORS. VS THE STATE (GOVT....,02-12-15,https://dhccaseinfo.nic.in/jupload/dhc/SKT/jud...


In [121]:
#Spliting the Party name column into party name and respondent's 
df['PARTY NAME'], df['RESPONDENT"S'] = df['PARTY NAME'].str.split(" VS ", 1).str 

In [145]:
# Reording the columns name
df = df[['S.NO', 'CASE NO', 'CATEGORY', 'PARTY NAME','RESPONDENT"S','JUDGEMENT DATE', 'JUDGEMENT PDF LINK']]

In [146]:
df.sample()

,S.NO,CASE NO,CATEGORY,PARTY NAME,"RESPONDENT""S",JUDGEMENT DATE,JUDGEMENT PDF LINK
10662,10662,CRL.L.P. 504/2011,Against acquittal,VIRENDER SINGH,DEEPAK BHATIA,08-04-13,https://dhccaseinfo.nic.in/jupload/dhc/GPM/jud...


In [147]:
df.to_csv("C:/Users/ADMIN/Desktop/New folder/Project/Court scrape/Court_data_final.csv", encoding='utf-8', index=False)